In [3]:
import cv2
import numpy as np
import os
import shutil
def calculate_contrasts(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    
    I_min, I_max = np.min(gray_image), np.max(gray_image)
    gray_image = (gray_image - I_min) / (I_max - I_min) # if (I_max - I_min)  != 0 else gray_image

    I_max = np.max(gray_image)
    I_min = np.min(gray_image)
    I_mean = np.mean(gray_image)

    if I_max > 1 or I_min > 1 or I_mean > 1:
        print(f"Warning: Image data range is not in [0, 1]. Max: {I_max}, Min: {I_min}, Mean: {I_mean}")


    rms_contrast = np.sqrt(np.mean((gray_image - I_mean)**2))
    michelson_contrast = (I_max - I_min) / (I_max + I_min)
    weber_contrast = np.mean((gray_image - I_mean) / I_mean)
    contrast_ratio = I_max / I_min if I_min != 0 else 0  # Avoid division by zero

    # Convert to 8-bit before calculating histogram
    gray_image_8bit = (gray_image * 255).astype(np.uint8)

    # Calculate entropy
    hist = cv2.calcHist([gray_image_8bit], [0], None, [256], [0,256])
    hist /= hist.sum()
    entropy = -np.sum(hist*np.log2(hist + np.finfo(float).eps))  # added epsilon to avoid log(0)

    return rms_contrast, michelson_contrast, weber_contrast, contrast_ratio, entropy

def process_images(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    rms_contrast_values = []
    michelson_contrast_values = []
    weber_contrast_values = []
    contrast_ratio_values = []
    entropy_values = []

    for root, dirs, files in os.walk(input_directory):
        for file in files:
            file_path = os.path.join(root, file)
            if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                upsampled_image = cv2.imread(file_path)

                # Upsample to 256x256
                upsampled_image = cv2.resize(upsampled_image, (128, 128), interpolation=cv2.INTER_LINEAR)

                rms_contrast, michelson_contrast, weber_contrast, contrast_ratio, entropy = calculate_contrasts(upsampled_image)
                entropy_values.append(entropy)
                rms_contrast_values.append(rms_contrast)
                michelson_contrast_values.append(michelson_contrast)
                weber_contrast_values.append(weber_contrast)
                contrast_ratio_values.append(contrast_ratio)

                # Create new filename with all contrast metrics
                filename, ext = os.path.splitext(file)
                new_filename = f"{filename}_RMS_{rms_contrast:.2f}_Michelson_{michelson_contrast:.2f}_Weber_{weber_contrast:.2f}_CR_{contrast_ratio:.2f}_Entropy_{entropy:.2f}{ext}"
                new_file_path = os.path.join(output_directory, new_filename)

                # Save the upsampled image file to the new directory
                cv2.imwrite(new_file_path, upsampled_image)

    # Print the average of each contrast metric
    avg_rms_contrast = np.mean(rms_contrast_values)
    std_rms_contrast = np.std(rms_contrast_values)
    avg_michelson_contrast = np.mean(michelson_contrast_values)
    avg_weber_contrast = np.mean(weber_contrast_values)
    avg_contrast_ratio = np.mean(contrast_ratio_values)
    avg_entropy = np.mean(entropy_values)
    std_entropy = np.std(entropy_values)

    print(f"Average RMS Contrast: {avg_rms_contrast:.4f}")
    print(f"STD RMS Contrast: {std_rms_contrast:.4f}")
    print(f"Average Michelson Contrast: {avg_michelson_contrast:.2f}")
    print(f"Average Weber Contrast: {avg_weber_contrast:.2f}")
    print(f"Average Contrast Ratio: {avg_contrast_ratio:.2f}")
    print(f"Average Entropy: {avg_entropy:.4f}")
    print(f"STD Entropy: {std_entropy:.4f}")

if __name__ == "__main__":
    #input_directory = '/home/alingold/pix2pix/results/combined_plant_0bp_noBlurryRemoval_NoGan_oldtest/test_latest/real_B'
    #output_directory = '/home/alingold/pix2pix/results/combined_plant_0bp_noBlurryRemoval_NoGan_oldtest/test_latest/contrast'

    #input_directory = '/home/alingold/pix2pix/CCM_project/dataset/normalized_random_100_images_pro_ccm'
    #output_directory = '/home/alingold/pix2pix/CCM_project/dataset/normalized_random_100_images_pro_ccm/contrast'

    #input_directory = '/home/alingold/pix2pix/CCM_project/dataset/normalized_random_100_images'
    #output_directory = '/home/alingold/pix2pix/CCM_project/dataset/normalized_random_100_images/contrast'

    input_directory = '/home/alingold/Training/optrode_cross_section_plant/normalized_random_100_images_ref'
    output_directory = '/home/alingold/Training/optrode_cross_section_plant/normalized_random_100_images_ref/contrast'
    
    process_images(input_directory, output_directory)


Average RMS Contrast: 0.2127
STD RMS Contrast: 0.0314
Average Michelson Contrast: 1.00
Average Weber Contrast: 0.00
Average Contrast Ratio: 0.00
Average Entropy: 5.4262
STD Entropy: 1.0579
